In [2]:
import json
import plotly.express as px
import pandas as pd
from datetime import datetime
import numpy as np
import json
import warnings
import geojson
import plotly.graph_objs as go
from prophet import Prophet
warnings.filterwarnings("ignore")

## Data Exploration

### Our World in Data

In [63]:
df = pd.read_csv('https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv')
df.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,AFG,Asia,Afghanistan,2020-02-24,5.0,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2020-02-25,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2020-02-26,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2020-02-27,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,2020-02-28,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN


In [64]:
df.columns

Index(['iso_code', 'continent', 'location', 'date', 'total_cases', 'new_cases',
       'new_cases_smoothed', 'total_deaths', 'new_deaths',
       'new_deaths_smoothed', 'total_cases_per_million',
       'new_cases_per_million', 'new_cases_smoothed_per_million',
       'total_deaths_per_million', 'new_deaths_per_million',
       'new_deaths_smoothed_per_million', 'reproduction_rate', 'icu_patients',
       'icu_patients_per_million', 'hosp_patients',
       'hosp_patients_per_million', 'weekly_icu_admissions',
       'weekly_icu_admissions_per_million', 'weekly_hosp_admissions',
       'weekly_hosp_admissions_per_million', 'new_tests', 'total_tests',
       'total_tests_per_thousand', 'new_tests_per_thousand',
       'new_tests_smoothed', 'new_tests_smoothed_per_thousand',
       'positive_rate', 'tests_per_case', 'tests_units', 'total_vaccinations',
       'people_vaccinated', 'people_fully_vaccinated', 'total_boosters',
       'new_vaccinations', 'new_vaccinations_smoothed',
       't

The dataframe contains columns which are not necassary. Therefore these will be dropped.

In [65]:
# Only consider relevant columns
df = df[['iso_code','date','location', 'continent','new_cases_per_million', 'total_cases','new_deaths_per_million','total_deaths']]

* iso_code for identifing a certain location and for using it in a map
* location represents the country
* new_cases_per_million gibt die täglich bestätigten neuen Fallzahlen an pro 1 Million Einwohner
* total_cases Aufsummierte Anzahl an bestätigten Fallzahlen
* new_deaths_per_million gibt die täglich bestätigten neuen Todesfälle an pro 1 Million Einwohner
* total_deaths Aufsummierte Anzahl an bestätigten Todesfällen

#### Nan Values

In [66]:
pd.DataFrame(df.isna().sum()).transpose()

,iso_code,date,location,continent,new_cases_per_million,total_cases,new_deaths_per_million,total_deaths
0,0,0,0,9267,3426,2666,20468,19948


Der Datensatz enthält keine fehlenden Werte in den Spalten iso_code, date und location weshalb keine Bereinigung stattfinden muss. Die fehlenden Werten in den numerischen Spalten resultieren daraus, dass keine Daten gemeldet wurden. Sie können daher einfach ignoriert werden.

#### Check unique values 

In [67]:
df.iso_code.nunique() , df.location.nunique()

(238, 238)

Jedem Land ist ein Iso-Code zugeordnet, mit dem dieses eindeutig identifiziert werden kann. Der Iso-Code wird benötigt um beispielsweise Chloropleth-Maps zu erstellen.

#### Check erster Corona-Fall

In [68]:
min(df[df['new_cases_per_million']>0].date)

'2020-01-23'

In [69]:
px.bar(df[df['new_cases_per_million']>0].groupby('date').sum().reset_index(),x="date",y="new_cases_per_million",template="plotly_white")

### DIVI Intensivregister

In [70]:
divi = pd.read_csv("https://diviexchange.blob.core.windows.net/%24web/zeitreihe-tagesdaten.csv")
divi.head()

,date,bundesland,gemeindeschluessel,anzahl_standorte,anzahl_meldebereiche,faelle_covid_aktuell,faelle_covid_aktuell_invasiv_beatmet,betten_frei,betten_belegt,betten_belegt_nur_erwachsen,betten_frei_nur_erwachsen
0,2020-04-24,1,1001,2,2,1,1,40,38,38,40
1,2020-04-24,1,1002,3,4,4,3,93,108,108,93
2,2020-04-24,1,1003,2,2,4,4,109,108,108,109
3,2020-04-24,1,1004,1,1,1,1,17,21,21,17
4,2020-04-24,1,1051,1,1,3,1,47,25,25,47


In [71]:
divi.columns

Index(['date', 'bundesland', 'gemeindeschluessel', 'anzahl_standorte',
       'anzahl_meldebereiche', 'faelle_covid_aktuell',
       'faelle_covid_aktuell_invasiv_beatmet', 'betten_frei', 'betten_belegt',
       'betten_belegt_nur_erwachsen', 'betten_frei_nur_erwachsen'],
      dtype='object')

In [72]:
# Es werden nur relevante Spalten betrachtet
divi = divi[['date', 'bundesland', 'gemeindeschluessel',  'faelle_covid_aktuell',
       'faelle_covid_aktuell_invasiv_beatmet', 'betten_frei', 'betten_belegt']]

Der Datensatz enthält die täglichen Meldungen an Corona Infizierten, invasis beatmeten Infizierten sowie die Bettenbelegung je Landkreis.
In der CSV-Datei sind die Kreise und Bundesländer nach dem Amtlichen Gemeindeschlüssel kodiert.
- 01 Schleswig-Holstein
- 02 Hamburg
- 03 Niedersachsen
- 04 Bremen
- 05 Nordrhein-Westfalen
- 06 Hessen
- 07 Rheinland-Pfalz
- 08 Baden-Württemberg
- 09 Bayern
- 10 Saarland
- 11 Berlin
- 12 Brandenburg
- 13 Mecklenburg-Vorpommern
- 14 Sachsen
- 15 Sachsen-Anhalt
- 16 Thüringen

Um die jeweiligen Landkreise auch namentlich identifizieren zu können, müssen diese im nächsten Schritt über Geodaten angereichert werden. 

In [73]:
print('Anzahl Bundesländer im Datensatz:', divi.bundesland.nunique())

Anzahl Bundesländer im Datensatz: 16


In [74]:
divi.describe()

,bundesland,gemeindeschluessel,faelle_covid_aktuell,faelle_covid_aktuell_invasiv_beatmet,betten_frei,betten_belegt
count,249538.000000,249538.000000,249538.000000,249538.000000,249538.000000,249538.000000
mean,7.973780,8298.554625,5.686781,3.233576,15.526184,54.016230
std,3.817755,3775.741870,13.798412,8.903898,22.668728,83.981024
min,1.000000,1001.000000,0.000000,0.000000,0.000000,0.000000
25%,5.000000,5758.000000,0.000000,0.000000,4.000000,13.000000
50%,8.000000,8231.000000,2.000000,1.000000,9.000000,29.000000
75%,9.000000,9677.000000,6.000000,3.000000,19.000000,61.000000
max,16.000000,16077.000000,458.000000,307.000000,485.000000,1157.000000


### RKI

In [75]:
rki = pd.read_csv("https://raw.githubusercontent.com/robert-koch-institut/COVID-19-Impfungen_in_Deutschland/master/Aktuell_Deutschland_Landkreise_COVID-19-Impfungen.csv")
rki.head()

,Impfdatum,LandkreisId_Impfort,Altersgruppe,Impfschutz,Anzahl
0,2020-12-27,1003,18-59,1,9
1,2020-12-27,1003,60+,1,19
2,2020-12-27,1053,18-59,1,13
3,2020-12-27,1053,60+,1,200
4,2020-12-27,1055,18-59,1,20


- LandkreisId_Impfort: Identifikationsnummer des Landkreises basierend auf dem Amtlichen Gemeindeschlüssel (AGS). Impfungen des Bundesressorts werden separat ausgewiesen, da die Impfstellen des Bundes ohne exakte Angabe des Impfortes melden.
- Altersgruppe: Altersgruppen der in der Impfgruppe enthaltenen Fälle nach Schema der KBV
- Impfschutz: 1: Unvollständiger Impfschutz, 2: Vollständiger Impfschutz, 3: Aufgefrischter Impfschutz

In [76]:
px.histogram(rki.groupby(['Impfdatum','Impfschutz']).sum().reset_index(),'Impfdatum', 'Anzahl',color='Impfschutz')

In [77]:
px.histogram(rki.groupby(['Impfdatum','Altersgruppe','Impfschutz']).sum().reset_index(),'Impfdatum', 'Anzahl',color='Altersgruppe')